In [326]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

In [327]:
job_data = pd.read_csv('job_descriptions.csv', nrows=20000)
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 Years,M.Tech,$59K-$99K,Douglas,Isle of Man,54.2361,-4.5481,Intern,26801,...,001-381-930-7517x737,Digital Marketing Specialist,Social Media Manager,Snagajob,Social Media Managers oversee an organizations...,"{'Flexible Spending Accounts (FSAs), Relocatio...","Social media platforms (e.g., Facebook, Twitte...","Manage and grow social media accounts, create ...",Icahn Enterprises,"{""Sector"":""Diversified"",""Industry"":""Diversifie..."
1,398454096642776,2 to 12 Years,BCA,$56K-$116K,Ashgabat,Turkmenistan,38.9697,59.5563,Intern,100340,...,461-509-4216,Web Developer,Frontend Web Developer,Idealist,Frontend Web Developers design and implement u...,"{'Health Insurance, Retirement Plans, Paid Tim...","HTML, CSS, JavaScript Frontend frameworks (e.g...","Design and code user interfaces for websites, ...",PNC Financial Services Group,"{""Sector"":""Financial Services"",""Industry"":""Com..."
2,481640072963533,0 to 12 Years,PhD,$61K-$104K,Macao,"Macao SAR, China",22.1987,113.5439,Temporary,84525,...,9687619505,Operations Manager,Quality Control Manager,Jobs2Careers,Quality Control Managers establish and enforce...,"{'Legal Assistance, Bonuses and Incentive Prog...",Quality control processes and methodologies St...,Establish and enforce quality control standard...,United Services Automobile Assn.,"{""Sector"":""Insurance"",""Industry"":""Insurance: P..."
3,688192671473044,4 to 11 Years,PhD,$65K-$91K,Porto-Novo,Benin,9.3077,2.3158,Full-Time,129896,...,+1-820-643-5431x47576,Network Engineer,Wireless Network Engineer,FlexJobs,"Wireless Network Engineers design, implement, ...","{'Transportation Benefits, Professional Develo...",Wireless network design and architecture Wi-Fi...,"Design, configure, and optimize wireless netwo...",Hess,"{""Sector"":""Energy"",""Industry"":""Mining, Crude-O..."
4,117057806156508,1 to 12 Years,MBA,$64K-$87K,Santiago,Chile,-35.6751,-71.5429,Intern,53944,...,343.975.4702x9340,Event Manager,Conference Manager,Jobs2Careers,A Conference Manager coordinates and manages c...,"{'Flexible Spending Accounts (FSAs), Relocatio...",Event planning Conference logistics Budget man...,Specialize in conference and convention planni...,Cairn Energy,"{""Sector"":""Energy"",""Industry"":""Energy - Oil & ..."


In [328]:
job_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Job Id            20000 non-null  int64  
 1   Experience        20000 non-null  object 
 2   Qualifications    20000 non-null  object 
 3   Salary Range      20000 non-null  object 
 4   location          20000 non-null  object 
 5   Country           20000 non-null  object 
 6   latitude          20000 non-null  float64
 7   longitude         20000 non-null  float64
 8   Work Type         20000 non-null  object 
 9   Company Size      20000 non-null  int64  
 10  Job Posting Date  20000 non-null  object 
 11  Preference        20000 non-null  object 
 12  Contact Person    20000 non-null  object 
 13  Contact           20000 non-null  object 
 14  Job Title         20000 non-null  object 
 15  Role              20000 non-null  object 
 16  Job Portal        20000 non-null  object

In [329]:
job_data.isna().sum()

Job Id               0
Experience           0
Qualifications       0
Salary Range         0
location             0
Country              0
latitude             0
longitude            0
Work Type            0
Company Size         0
Job Posting Date     0
Preference           0
Contact Person       0
Contact              0
Job Title            0
Role                 0
Job Portal           0
Job Description      0
Benefits             0
skills               0
Responsibilities     0
Company              0
Company Profile     66
dtype: int64

In [330]:
job_data.columns

Index(['Job Id', 'Experience', 'Qualifications', 'Salary Range', 'location',
       'Country', 'latitude', 'longitude', 'Work Type', 'Company Size',
       'Job Posting Date', 'Preference', 'Contact Person', 'Contact',
       'Job Title', 'Role', 'Job Portal', 'Job Description', 'Benefits',
       'skills', 'Responsibilities', 'Company', 'Company Profile'],
      dtype='object')

In [331]:
job_data.dropna(inplace=True)
job_data.isna().sum()

Job Id              0
Experience          0
Qualifications      0
Salary Range        0
location            0
Country             0
latitude            0
longitude           0
Work Type           0
Company Size        0
Job Posting Date    0
Preference          0
Contact Person      0
Contact             0
Job Title           0
Role                0
Job Portal          0
Job Description     0
Benefits            0
skills              0
Responsibilities    0
Company             0
Company Profile     0
dtype: int64

In [332]:
for col in job_data.columns:
    if job_data[col].dtype == 'object':
        job_data[col] = job_data[col].str.lower().str.strip()

In [333]:
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Contact,Job Title,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile
0,1089843540111562,5 to 15 years,m.tech,$59k-$99k,douglas,isle of man,54.2361,-4.5481,intern,26801,...,001-381-930-7517x737,digital marketing specialist,social media manager,snagajob,social media managers oversee an organizations...,"{'flexible spending accounts (fsas), relocatio...","social media platforms (e.g., facebook, twitte...","manage and grow social media accounts, create ...",icahn enterprises,"{""sector"":""diversified"",""industry"":""diversifie..."
1,398454096642776,2 to 12 years,bca,$56k-$116k,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,461-509-4216,web developer,frontend web developer,idealist,frontend web developers design and implement u...,"{'health insurance, retirement plans, paid tim...","html, css, javascript frontend frameworks (e.g...","design and code user interfaces for websites, ...",pnc financial services group,"{""sector"":""financial services"",""industry"":""com..."
2,481640072963533,0 to 12 years,phd,$61k-$104k,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,9687619505,operations manager,quality control manager,jobs2careers,quality control managers establish and enforce...,"{'legal assistance, bonuses and incentive prog...",quality control processes and methodologies st...,establish and enforce quality control standard...,united services automobile assn.,"{""sector"":""insurance"",""industry"":""insurance: p..."
3,688192671473044,4 to 11 years,phd,$65k-$91k,porto-novo,benin,9.3077,2.3158,full-time,129896,...,+1-820-643-5431x47576,network engineer,wireless network engineer,flexjobs,"wireless network engineers design, implement, ...","{'transportation benefits, professional develo...",wireless network design and architecture wi-fi...,"design, configure, and optimize wireless netwo...",hess,"{""sector"":""energy"",""industry"":""mining, crude-o..."
4,117057806156508,1 to 12 years,mba,$64k-$87k,santiago,chile,-35.6751,-71.5429,intern,53944,...,343.975.4702x9340,event manager,conference manager,jobs2careers,a conference manager coordinates and manages c...,"{'flexible spending accounts (fsas), relocatio...",event planning conference logistics budget man...,specialize in conference and convention planni...,cairn energy,"{""sector"":""energy"",""industry"":""energy - oil & ..."


In [334]:
job_data['Job Posting Date'] = pd.to_datetime(job_data['Job Posting Date'])
job_data['Job Posting Month'] = job_data['Job Posting Date'].dt.month
job_data['Job Posting Year'] = job_data['Job Posting Date'].dt.year

In [335]:
cols_to_drop = ['Contact', 'Contact Person'] 

job_data.drop(cols_to_drop, axis=1, inplace=True)
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Role,Job Portal,Job Description,Benefits,skills,Responsibilities,Company,Company Profile,Job Posting Month,Job Posting Year
0,1089843540111562,5 to 15 years,m.tech,$59k-$99k,douglas,isle of man,54.2361,-4.5481,intern,26801,...,social media manager,snagajob,social media managers oversee an organizations...,"{'flexible spending accounts (fsas), relocatio...","social media platforms (e.g., facebook, twitte...","manage and grow social media accounts, create ...",icahn enterprises,"{""sector"":""diversified"",""industry"":""diversifie...",4,2022
1,398454096642776,2 to 12 years,bca,$56k-$116k,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,frontend web developer,idealist,frontend web developers design and implement u...,"{'health insurance, retirement plans, paid tim...","html, css, javascript frontend frameworks (e.g...","design and code user interfaces for websites, ...",pnc financial services group,"{""sector"":""financial services"",""industry"":""com...",12,2022
2,481640072963533,0 to 12 years,phd,$61k-$104k,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,quality control manager,jobs2careers,quality control managers establish and enforce...,"{'legal assistance, bonuses and incentive prog...",quality control processes and methodologies st...,establish and enforce quality control standard...,united services automobile assn.,"{""sector"":""insurance"",""industry"":""insurance: p...",9,2022
3,688192671473044,4 to 11 years,phd,$65k-$91k,porto-novo,benin,9.3077,2.3158,full-time,129896,...,wireless network engineer,flexjobs,"wireless network engineers design, implement, ...","{'transportation benefits, professional develo...",wireless network design and architecture wi-fi...,"design, configure, and optimize wireless netwo...",hess,"{""sector"":""energy"",""industry"":""mining, crude-o...",2,2023
4,117057806156508,1 to 12 years,mba,$64k-$87k,santiago,chile,-35.6751,-71.5429,intern,53944,...,conference manager,jobs2careers,a conference manager coordinates and manages c...,"{'flexible spending accounts (fsas), relocatio...",event planning conference logistics budget man...,specialize in conference and convention planni...,cairn energy,"{""sector"":""energy"",""industry"":""energy - oil & ...",10,2022


In [336]:
job_data['Salary Range'] = job_data['Salary Range'].str.replace('k', '000').str.replace('$', '')
job_data['Salary Range']

0         59000-99000
1        56000-116000
2        61000-104000
3         65000-91000
4         64000-87000
             ...     
19995    62000-109000
19996    61000-119000
19997    63000-111000
19998    62000-129000
19999     60000-88000
Name: Salary Range, Length: 19934, dtype: object

In [337]:
job_data['Min Salary'] = job_data['Salary Range'].str.split('-').str[0].astype(float)
job_data['Max Salary'] = job_data['Salary Range'].str.split('-').str[1].astype(float)
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Description,Benefits,skills,Responsibilities,Company,Company Profile,Job Posting Month,Job Posting Year,Min Salary,Max Salary
0,1089843540111562,5 to 15 years,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,social media managers oversee an organizations...,"{'flexible spending accounts (fsas), relocatio...","social media platforms (e.g., facebook, twitte...","manage and grow social media accounts, create ...",icahn enterprises,"{""sector"":""diversified"",""industry"":""diversifie...",4,2022,59000.0,99000.0
1,398454096642776,2 to 12 years,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,frontend web developers design and implement u...,"{'health insurance, retirement plans, paid tim...","html, css, javascript frontend frameworks (e.g...","design and code user interfaces for websites, ...",pnc financial services group,"{""sector"":""financial services"",""industry"":""com...",12,2022,56000.0,116000.0
2,481640072963533,0 to 12 years,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,quality control managers establish and enforce...,"{'legal assistance, bonuses and incentive prog...",quality control processes and methodologies st...,establish and enforce quality control standard...,united services automobile assn.,"{""sector"":""insurance"",""industry"":""insurance: p...",9,2022,61000.0,104000.0
3,688192671473044,4 to 11 years,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,"wireless network engineers design, implement, ...","{'transportation benefits, professional develo...",wireless network design and architecture wi-fi...,"design, configure, and optimize wireless netwo...",hess,"{""sector"":""energy"",""industry"":""mining, crude-o...",2,2023,65000.0,91000.0
4,117057806156508,1 to 12 years,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,a conference manager coordinates and manages c...,"{'flexible spending accounts (fsas), relocatio...",event planning conference logistics budget man...,specialize in conference and convention planni...,cairn energy,"{""sector"":""energy"",""industry"":""energy - oil & ...",10,2022,64000.0,87000.0


In [338]:
job_data['Experience'] = job_data['Experience'].str.replace('years', '')
job_data.head() 

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Description,Benefits,skills,Responsibilities,Company,Company Profile,Job Posting Month,Job Posting Year,Min Salary,Max Salary
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,social media managers oversee an organizations...,"{'flexible spending accounts (fsas), relocatio...","social media platforms (e.g., facebook, twitte...","manage and grow social media accounts, create ...",icahn enterprises,"{""sector"":""diversified"",""industry"":""diversifie...",4,2022,59000.0,99000.0
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,frontend web developers design and implement u...,"{'health insurance, retirement plans, paid tim...","html, css, javascript frontend frameworks (e.g...","design and code user interfaces for websites, ...",pnc financial services group,"{""sector"":""financial services"",""industry"":""com...",12,2022,56000.0,116000.0
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,quality control managers establish and enforce...,"{'legal assistance, bonuses and incentive prog...",quality control processes and methodologies st...,establish and enforce quality control standard...,united services automobile assn.,"{""sector"":""insurance"",""industry"":""insurance: p...",9,2022,61000.0,104000.0
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,"wireless network engineers design, implement, ...","{'transportation benefits, professional develo...",wireless network design and architecture wi-fi...,"design, configure, and optimize wireless netwo...",hess,"{""sector"":""energy"",""industry"":""mining, crude-o...",2,2023,65000.0,91000.0
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,a conference manager coordinates and manages c...,"{'flexible spending accounts (fsas), relocatio...",event planning conference logistics budget man...,specialize in conference and convention planni...,cairn energy,"{""sector"":""energy"",""industry"":""energy - oil & ...",10,2022,64000.0,87000.0


In [339]:
job_data['Min Exp'] = job_data['Experience'].str.split('to').str[0].astype(float)
job_data['Max Exp'] = job_data['Experience'].str.split('to').str[1].astype(float)
job_data['Avg Exp'] = (job_data['Min Exp'] + job_data['Max Exp']) / 2

job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Responsibilities,Company,Company Profile,Job Posting Month,Job Posting Year,Min Salary,Max Salary,Min Exp,Max Exp,Avg Exp
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,"manage and grow social media accounts, create ...",icahn enterprises,"{""sector"":""diversified"",""industry"":""diversifie...",4,2022,59000.0,99000.0,5.0,15.0,10.0
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,"design and code user interfaces for websites, ...",pnc financial services group,"{""sector"":""financial services"",""industry"":""com...",12,2022,56000.0,116000.0,2.0,12.0,7.0
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,establish and enforce quality control standard...,united services automobile assn.,"{""sector"":""insurance"",""industry"":""insurance: p...",9,2022,61000.0,104000.0,0.0,12.0,6.0
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,"design, configure, and optimize wireless netwo...",hess,"{""sector"":""energy"",""industry"":""mining, crude-o...",2,2023,65000.0,91000.0,4.0,11.0,7.5
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,specialize in conference and convention planni...,cairn energy,"{""sector"":""energy"",""industry"":""energy - oil & ...",10,2022,64000.0,87000.0,1.0,12.0,6.5


In [340]:
# import ast
# job_data['Company Profile'] = job_data['Company Profile'].apply(lambda x: ast.literal_eval(x))

In [341]:
job_data['Company Profile'] = job_data['Company Profile'].str.replace('"duke"', 'duke')

In [342]:
job_data['Company Profile'] = job_data['Company Profile'].apply(lambda x: ast.literal_eval(x))

In [343]:
job_data.head() 

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Responsibilities,Company,Company Profile,Job Posting Month,Job Posting Year,Min Salary,Max Salary,Min Exp,Max Exp,Avg Exp
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,"manage and grow social media accounts, create ...",icahn enterprises,"{'sector': 'diversified', 'industry': 'diversi...",4,2022,59000.0,99000.0,5.0,15.0,10.0
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,"design and code user interfaces for websites, ...",pnc financial services group,"{'sector': 'financial services', 'industry': '...",12,2022,56000.0,116000.0,2.0,12.0,7.0
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,establish and enforce quality control standard...,united services automobile assn.,"{'sector': 'insurance', 'industry': 'insurance...",9,2022,61000.0,104000.0,0.0,12.0,6.0
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,"design, configure, and optimize wireless netwo...",hess,"{'sector': 'energy', 'industry': 'mining, crud...",2,2023,65000.0,91000.0,4.0,11.0,7.5
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,specialize in conference and convention planni...,cairn energy,"{'sector': 'energy', 'industry': 'energy - oil...",10,2022,64000.0,87000.0,1.0,12.0,6.5


In [344]:
job_data['Sector'] = job_data['Company Profile'].apply(lambda x: x.get('sector')) 
job_data['Industry'] = job_data['Company Profile'].apply(lambda x: x.get('industry'))
job_data['City'] = job_data['Company Profile'].apply(lambda x: x.get('city'))
job_data['State'] = job_data['Company Profile'].apply(lambda x: x.get('state'))
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Job Posting Year,Min Salary,Max Salary,Min Exp,Max Exp,Avg Exp,Sector,Industry,City,State
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,2022,59000.0,99000.0,5.0,15.0,10.0,diversified,diversified financials,sunny isles beach,florida
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,2022,56000.0,116000.0,2.0,12.0,7.0,financial services,commercial banks,pittsburgh,pennsylvania
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,2022,61000.0,104000.0,0.0,12.0,6.0,insurance,insurance: property and casualty (stock),san antonio,texas
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,2023,65000.0,91000.0,4.0,11.0,7.5,energy,"mining, crude-oil production",new york,new york
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,2022,64000.0,87000.0,1.0,12.0,6.5,energy,energy - oil & gas exploration & production,edinburgh,scotland


In [345]:
job_data['Job Description Length'] = job_data['Job Description'].str.len()
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Min Salary,Max Salary,Min Exp,Max Exp,Avg Exp,Sector,Industry,City,State,Job Description Length
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,59000.0,99000.0,5.0,15.0,10.0,diversified,diversified financials,sunny isles beach,florida,200
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,56000.0,116000.0,2.0,12.0,7.0,financial services,commercial banks,pittsburgh,pennsylvania,231
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,61000.0,104000.0,0.0,12.0,6.0,insurance,insurance: property and casualty (stock),san antonio,texas,222
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,65000.0,91000.0,4.0,11.0,7.5,energy,"mining, crude-oil production",new york,new york,200
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,64000.0,87000.0,1.0,12.0,6.5,energy,energy - oil & gas exploration & production,edinburgh,scotland,235


In [346]:
job_data['Benefits Count'] = job_data['Benefits'].apply(lambda x: len(x.split(',')))
job_data.head()

,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,Max Salary,Min Exp,Max Exp,Avg Exp,Sector,Industry,City,State,Job Description Length,Benefits Count
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,99000.0,5.0,15.0,10.0,diversified,diversified financials,sunny isles beach,florida,200,5
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,116000.0,2.0,12.0,7.0,financial services,commercial banks,pittsburgh,pennsylvania,231,5
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,104000.0,0.0,12.0,6.0,insurance,insurance: property and casualty (stock),san antonio,texas,222,5
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,91000.0,4.0,11.0,7.5,energy,"mining, crude-oil production",new york,new york,200,5
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,87000.0,1.0,12.0,6.5,energy,energy - oil & gas exploration & production,edinburgh,scotland,235,5


In [347]:
job_data['Benefits'].head() 

0    {'flexible spending accounts (fsas), relocatio...
1    {'health insurance, retirement plans, paid tim...
2    {'legal assistance, bonuses and incentive prog...
3    {'transportation benefits, professional develo...
4    {'flexible spending accounts (fsas), relocatio...
Name: Benefits, dtype: object

In [348]:
unique_benefits = {}
for indx, benefit_string in enumerate(job_data['Benefits']):
    benefit_string = re.sub(r"[^a-zA-Z0-9 ,]", "", benefit_string)
    benefit_list = [benefit.strip() for benefit in benefit_string.split(",")]

    for benefit in benefit_list:
        job_data.loc[job_data.index[indx], benefit] = 1
        if benefit in unique_benefits.keys():
            unique_benefits[benefit] += 1
        else:
            unique_benefits[benefit] = 1
 
print("Unique Benefits: ", len(unique_benefits))
job_data.head()

Unique Benefits:  25


,Job Id,Experience,Qualifications,Salary Range,location,Country,latitude,longitude,Work Type,Company Size,...,profitsharing,life and disability insurance,stock options or equity grants,social and recreational activities,tuition reimbursement,parental leave,employee referral programs,health and wellness facilities,casual dress code,childcare assistance
0,1089843540111562,5 to 15,m.tech,59000-99000,douglas,isle of man,54.2361,-4.5481,intern,26801,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,398454096642776,2 to 12,bca,56000-116000,ashgabat,turkmenistan,38.9697,59.5563,intern,100340,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,481640072963533,0 to 12,phd,61000-104000,macao,"macao sar, china",22.1987,113.5439,temporary,84525,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,688192671473044,4 to 11,phd,65000-91000,porto-novo,benin,9.3077,2.3158,full-time,129896,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,117057806156508,1 to 12,mba,64000-87000,santiago,chile,-35.6751,-71.5429,intern,53944,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [349]:
# for skill_string in job_data['skills']:
#     # print(skill_string)
#     while re.search(r'\([^)]*\)', skill_string):
#         skill_string = re.sub(r'\([^)]*\)', '', skill_string)
#     # print(skill_string)

In [350]:
# bins = [0, 50, 500, 5000, float('inf')]
# labels = ['Small', 'Medium', 'Large', 'Very Large']

# job_data['Company Size Category'] = pd.cut(job_data['Company Size'], bins=bins, labels=labels)
# job_data['Company Size Category'].value_counts()

In [351]:
# job_data.to_csv('job_data_sahy.csv')

In [352]:
unique_qualifications = {}
for indx, qualification_string in enumerate(job_data['Qualifications']):
    qualification_string = re.sub(r"[^a-zA-Z0-9 ,]", "", qualification_string)
    qualification_list = [qualification.strip() for qualification in qualification_string.split(",")]
            
    for qualification in qualification_list:
        job_data.loc[job_data.index[indx], qualification] = 1
        if qualification in unique_qualifications.keys():
            unique_qualifications[qualification] += 1
        else:
            unique_qualifications[qualification] = 1

print("Unique Qualifications: ", len(unique_qualifications))
unique_qualifications

Unique Qualifications:  10


{'mtech': 2008,
 'bca': 1946,
 'phd': 1930,
 'mba': 2049,
 'mca': 1964,
 'mcom': 1998,
 'bba': 1968,
 'btech': 2099,
 'bcom': 1955,
 'ba': 2017}

In [353]:
for key in unique_benefits.keys() + unique_qualifications.keys():
    job_data[key] =  job_data[key].fillna(0)

job_data.head()

TypeError: unsupported operand type(s) for +: 'dict_keys' and 'dict_keys'

In [ ]:
job_data['Work Type'] = job_data['Work Type'].astype('category')
job_data = pd.concat([pd.get_dummies(job_data, columns=['Work Type'], prefix='Work Type', prefix_sep='_', dummy_na=False), job_data['Work Type']], axis=1)
job_data.head()

In [ ]:
job_data['Male_Preference'] = job_data['Preference'].isin(['male', 'both'])
job_data['Female_Preference'] = job_data['Preference'].isin(['female', 'both'])
print(job_data['Male_Preference'].value_counts())
print(job_data['Female_Preference'].value_counts())

In [ ]:
job_data['Long Description'] = ""

des_cols = ['Job Description', 'Job Title', 'Role', 'skills', 'Responsibilities']

for col in des_cols:
    job_data['Long Description'] = job_data['Long Description'] + " " + job_data[col]
job_data['Long Description'] = job_data['Long Description'].str.strip()

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')

en_stopwords = stopwords.words('english')
en_stopwords.remove("not")
ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [ ]:
# def remove_stopwords(

# job_data['Long Description'] = job_data['Long Description'].apply(